# Número óptimo de Tópicos con métrica Coherence

In [1]:
import sys
sys.path.insert(0, '..')
from utils.cargar import df_caso
from utils.preprocesamiento import process_df, StemmerTokenizer
from sklearn.model_selection import train_test_split

tokenizador = StemmerTokenizer(stem=False,rmv_punctuation=True)

caso = 'laura'

df = df_caso(caso)
df = process_df(df,'comment','sel',verbose=True)
df = df.drop(columns=['user_id','team_id','gender','df','title','opt_left','opt_right','max_num','phase','time','curso'])

df_train, df_test, _, _ = train_test_split(df, df['sel'], test_size=.05, stratify=df['sel'], random_state=0)
tokenized_corpus = [tokenizador(document) for document in df_train['comment']]
tokenized_test = [tokenizador(document) for document in df_test['comment']]

549 rows found with non string elements for column comment (11.03%)
Deleting 362 columns for which max target value is over 7 (7.27%)
4067 available rows after processing


In [2]:
import gensim
from gensim import corpora

# Create a dictionary from the tokenized corpus
dictionary = corpora.Dictionary(tokenized_corpus)

# Convert the tokenized corpus into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_corpus]

In [3]:
from gensim.models import CoherenceModel

# Set the range of topic numbers to try
min_topics = 2
max_topics = 10
step_size = 1

# Initialize variables for best coherence score and best number of topics
best_coherence_score = -1
best_num_topics = -1

# Iterate over the range of topic numbers
for num_topics in range(min_topics, max_topics+1, step_size):
    # Train the LDA model
    lda_model = gensim.models.LdaModel(doc_term_matrix, num_topics=num_topics, id2word=dictionary, passes=10)
    
    # Calculate coherence score
    coherence_model = CoherenceModel(model=lda_model, texts=tokenized_corpus, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    
    # Check if coherence score is the best so far
    if coherence_score > best_coherence_score:
        best_coherence_score = coherence_score
        best_num_topics = num_topics

# Print the best number of topics
print(f"Best number of topics: {best_num_topics}")

Best number of topics: 3


In [4]:
%%time
lda_model_opt = gensim.models.LdaModel(doc_term_matrix, num_topics=best_num_topics, id2word=dictionary, passes=10)

CPU times: user 13.8 s, sys: 16.2 ms, total: 13.8 s
Wall time: 13.8 s


In [5]:
# Print the generated topics
topics = lda_model_opt.print_topics(num_topics=num_topics)
for topic in topics:
    print(topic)

(0, '0.025*"trabajo" + 0.024*"error" + 0.021*"tiempo" + 0.019*"familia" + 0.014*"errores" + 0.014*"si" + 0.014*"Laura" + 0.012*"vida" + 0.010*"adecuado" + 0.010*"debe"')
(1, '0.039*"proyecto" + 0.021*"si" + 0.012*"renunciar" + 0.010*"impacto" + 0.008*"bien" + 0.008*"mejor" + 0.007*"ambiente" + 0.007*"quedarse" + 0.007*"Si" + 0.007*"daño"')
(2, '0.018*"si" + 0.015*"proyecto" + 0.011*"bien" + 0.011*"trabajo" + 0.009*"Si" + 0.009*"mas" + 0.008*"ser" + 0.008*"vida" + 0.008*"parece" + 0.007*"renunciar"')


In [6]:
print(' '.join(tokenized_test[0]) + '\n')

# Convert the tokenized document into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(tokenized_test[0])]

# Get the topic probabilities for the new document
topic_probs = lda_model_opt.get_document_topics(doc_term_matrix)[0]

# Print the topic probabilities
for topic, prob in topic_probs:
    print(f"Topic {topic}: {prob}")

Personalmente encuentro importante vida personal trabajo dado pierden momentos realmente importan Sin embargo debe existir mínimo dedicación existir

Topic 0: 0.897127628326416
Topic 1: 0.08362531661987305
Topic 2: 0.019246986135840416
